# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fae0804c550>
├── 'a' --> tensor([[ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696]])
└── 'x' --> <FastTreeValue 0x7fae0804cf70>
    └── 'c' --> tensor([[ 0.3072, -1.1103, -0.6346, -0.7643],
                        [-0.0895,  0.1214,  0.2436,  1.0044],
                        [ 0.0679, -0.1757,  0.1821,  0.6198]])

In [4]:
t.a

tensor([[ 0.5409, -0.3825,  0.2127],
        [ 0.1611,  0.1480, -0.2696]])

In [5]:
%timeit t.a

64.6 ns ± 0.0257 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fae0804c550>
├── 'a' --> tensor([[ 1.1810,  0.3039,  0.9946],
│                   [-0.8064,  0.3696, -0.0132]])
└── 'x' --> <FastTreeValue 0x7fae0804cf70>
    └── 'c' --> tensor([[ 0.3072, -1.1103, -0.6346, -0.7643],
                        [-0.0895,  0.1214,  0.2436,  1.0044],
                        [ 0.0679, -0.1757,  0.1821,  0.6198]])

In [7]:
%timeit t.a = new_value

64.4 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696]]),
    x: Batch(
           c: tensor([[ 0.3072, -1.1103, -0.6346, -0.7643],
                      [-0.0895,  0.1214,  0.2436,  1.0044],
                      [ 0.0679, -0.1757,  0.1821,  0.6198]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5409, -0.3825,  0.2127],
        [ 0.1611,  0.1480, -0.2696]])

In [11]:
%timeit b.a

55.1 ns ± 0.0347 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3924, -0.3521, -0.3187],
               [-0.2780, -1.9680,  0.1789]]),
    x: Batch(
           c: tensor([[ 0.3072, -1.1103, -0.6346, -0.7643],
                      [-0.0895,  0.1214,  0.2436,  1.0044],
                      [ 0.0679, -0.1757,  0.1821,  0.6198]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.259 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 74.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 280 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 424 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fadf8fcde20>
├── 'a' --> tensor([[[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]],
│           
│                   [[ 0.5409, -0.3825,  0.2127],
│                    [ 0.1611,  0.1480, -0.2696]]])
└── 'x' --> <FastTreeValue 0x7fadf8fc6b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 59.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fad53295f40>
├── 'a' --> tensor([[ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696],
│                   [ 0.5409, -0.3825,  0.2127],
│                   [ 0.1611,  0.1480, -0.2696]])
└── 'x' --> <FastTreeValue 0x7fad53872b20>
    └── 'c' --> tensor([[ 0.3072, -1.1103, -0.6346, -0.7643],
                        [-0.0895,  0.1214,  0.2436,  1.0044],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 52.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.4 µs ± 91.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]],
       
               [[ 0.5409, -0.3825,  0.2127],
                [ 0.1611,  0.1480, -0.2696]]]),
    x: Batch(
           c: tensor([[[ 0.3072, -1.1103, -0.6346, -0.7643],
                       [-0.0895,  0.1214,  0.2436,  1.0044],
                       [ 0.0679, -0.1757,  0.1821,  0.6198]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 93.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696],
               [ 0.5409, -0.3825,  0.2127],
               [ 0.1611,  0.1480, -0.2696]]),
    x: Batch(
           c: tensor([[ 0.3072, -1.1103, -0.6346, -0.7643],
                      [-0.0895,  0.1214,  0.2436,  1.0044],
                      [ 0.0679, -0.1757,  0.1821,  0.6198],
                      [ 0.3072, -1.1103, -0.6346, -0.7643],
                      [-0.0895,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 462 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
